In [1]:
import pandas as pd
import numpy as np
import joblib
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression

In [2]:
DATA_PATH = '../data/train.csv' 
try:
    df = pd.read_csv(DATA_PATH)
except FileNotFoundError:
    print(f"ERROR: File {DATA_PATH} tidak ditemukan. Pastikan Anda menempatkan train.csv di folder data/.")


In [3]:
df.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


Preprocess Data

In [4]:
df['SalePrice'] = np.log1p(df['SalePrice'])
numeric_cols = df.select_dtypes(include=np.number).columns.tolist()
categorical_cols = df.select_dtypes(include='object').columns.tolist()

In [5]:
# Handle missing values
for col in numeric_cols:
    df[col] = df[col].fillna(df[col].mean())
for col in categorical_cols:
    df[col] = df[col].fillna('None')

In [6]:
# One-Hot Encoding
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
if 'Id' in df.columns:
        df = df.drop('Id', axis=1)

Split Data

In [7]:
# Split data
X = df.drop('SalePrice', axis=1)
y = df['SalePrice']

In [8]:
final_features = X.columns.tolist()

In [ ]:
joblib.dump(final_features, '../models/final_features.pkl')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"Data Training: {X_train.shape[0]} samples")
print(f"Data Testing: {X_test.shape[0]} samples")

Data Training: 1168 samples
Data Testing: 292 samples


Model Initialization dan Training

In [10]:
models = {
        "XGBoost_Final": XGBRegressor(n_estimators=1000, learning_rate=0.05, random_state=42),
        "RandomForest": RandomForestRegressor(n_estimators=100, random_state=42),
        "LinearRegression": LinearRegression()
    }
performance_metrics = {}

In [ ]:
# Train and evaluate models
for name, model in models.items():
        print(f"Melatih Model: {name}...")
        
        # Training Model
        model.fit(X_train, y_train)
        
        # Evaluasi
        y_pred = model.predict(X_test)
        mse_log = mean_squared_error(y_test, y_pred)
        rmse_log = np.sqrt(mse_log)
        r2 = r2_score(y_test, y_pred)

        # Metrik dalam Original Scale
        y_test_original = np.expm1(y_test)
        y_pred_original = np.expm1(y_pred)
        rmse_original = np.sqrt(mean_squared_error(y_test_original, y_pred_original))
        mae_original = mean_absolute_error(y_test_original, y_pred_original)

        # Menyimpan Model
        model_filename = f'../models/{name}.pkl'
        joblib.dump(model, model_filename)

        performance_metrics[name] = {
            'RMSE': float(rmse_original),
            'R2': float(r2),
            'MAE': float(mae_original)
        }
        print(f"  Model {name} disimpan di: {model_filename}")
        print(f'  RMSE: ${rmse_original:,.0f} | R2: {r2:.4f} | MAE: ${mae_original:,.0f}')

Melatih Model: XGBoost_Final...
  Model XGBoost_Final disimpan di: ../models/XGBoost_Final.pkl
  RMSE: $26,494 | R2: 0.8943 | MAE: $16,523
Melatih Model: RandomForest...
  Model RandomForest disimpan di: ../models/RandomForest.pkl
  RMSE: $28,779 | R2: 0.8866 | MAE: $17,450
Melatih Model: LinearRegression...
  Model LinearRegression disimpan di: ../models/LinearRegression.pkl
  RMSE: $25,457 | R2: 0.8382 | MAE: $15,511


In [ ]:
# Save metrics
joblib.dump(performance_metrics, '../models/model_metrics.pkl')
print('Metrics saved:', performance_metrics)

Metrics saved: {'XGBoost_Final': {'RMSE': 26493.99815940876, 'R2': 0.8942822852366451, 'MAE': 16522.62992294522}, 'RandomForest': {'RMSE': 28778.965307629423, 'R2': 0.8866490434370606, 'MAE': 17450.220488643452}, 'LinearRegression': {'RMSE': 25456.877026762737, 'R2': 0.8382198462946938, 'MAE': 15510.75231156695}}
